
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>

# 3.2 DEMO: Fine Grained Access Control using Dynamic Views and Partition Filtering [Recipient]

## Overview
This demonstration showcases how to consume data shared with two key data governance patterns:

- **Dynamic Views**: Row-level security based on recipient identity using `current_recipient()`
- **Partition Filtering**: Region-based data access control through table partitioning

**Provider Notebook:** Creates source data, implements one dynamic view example and one partition filtering example, then shares with recipients.

**Recipient Notebook (This Notebook):** Access shared views as the US partner recipient and observe how access controls work.

### Learning Objectives
By the end of this demo, you will be able to:
1. Mount and query shared data with dynamic access controls
2. Understand how recipient identity affects data visibility
3. Observe the difference between dynamic views and partition filtering

## Background

**Scenario:** You are the "US Partner" receiving customer order data from GlobalRetail Inc. The provider has sales data from multiple regions (United States, Europe, Asia Pacific) but you should only see data from the United States region through two different access control patterns.

## Setup

In [ ]:
%run ./Includes/Demo-Setup-3.2

## Example 1: Accessing Data via Dynamic Views

Dynamic views use the `current_recipient()` function to filter data based on who is accessing it. As the `us_partner` recipient, you will access the shared view and only see United States data.

In [ ]:
-- Step 1: Create catalog from the shared view
-- As 'us_partner', you receive access to the dynamic view
CREATE CATALOG IF NOT EXISTS shared_orders_dynamic
USING SHARE provider.regional_orders_dynamic
COMMENT 'Orders shared via dynamic view';

In [ ]:
-- Step 2: Explore the shared catalog
SHOW SCHEMAS IN shared_orders_dynamic;

In [ ]:
-- Step 3: Query the dynamic view
-- As 'us_partner', you'll only see United States orders
-- The view automatically filters based on current_recipient()
SELECT * 
FROM shared_orders_dynamic.sales.regional_orders_view
ORDER BY order_date;

### Step 4: Analyze Your Regional Data

In [ ]:
-- Verify you only see United States data
SELECT 
  region,
  COUNT(*) as order_count,
  SUM(amount) as total_revenue,
  AVG(amount) as avg_order_value
FROM shared_orders_dynamic.sales.regional_orders_view
GROUP BY region;

In [ ]:
**Key Observation:** 
- You only see orders from the United States region
- The filtering happens automatically based on your recipient identity (`us_partner`)
- Other partners accessing the same share see different data (their region only)
- The `current_recipient()` function returns `'us_partner'` when you query the view

## Example 2: Accessing Data via Partition Filtering

Partition filtering shares specific partitions of a table. As `us_partner`, you will receive a share containing only the United States partition.

In [ ]:
-- Step 1: Create catalog from the partition-specific share
-- As 'us_partner', you receive a share with only the United States partition
CREATE CATALOG IF NOT EXISTS us_orders_partition
USING SHARE provider.us_orders_share
COMMENT 'United States orders via partition filtering';

In [ ]:
-- Step 2: Explore the shared catalog
SHOW SCHEMAS IN us_orders_partition;

### Step 3: Query the Partitioned Table

In [ ]:
-- Query the partitioned table - you can only access the United States partition
SELECT * 
FROM us_orders_partition.sales.orders_by_region
ORDER BY order_date;

### Step 4: Verify Partition Access

In [ ]:
-- Verify you only have access to United States partition
SELECT 
  region,
  COUNT(*) as order_count,
  SUM(amount) as total_revenue
FROM us_orders_partition.sales.orders_by_region
GROUP BY region;

**Key Observation:**
- You only have access to the United States partition
- The data is physically separated at the storage level
- You cannot query or even see that other partitions (Europe, Asia Pacific) exist
- This provides storage-level isolation

In [ ]:
## Summary

✅ **What we accomplished as the US Partner:**

**Example 1 - Dynamic Views:**
- Mounted a shared catalog containing a dynamic view
- Queried the view and automatically received only United States data
- The `current_recipient()` function filtered rows based on our identity
- Other partners access the same share but see different data

**Example 2 - Partition Filtering:**
- Mounted a partition-specific share containing only United States data
- Queried the partitioned table with storage-level isolation
- Cannot access or see other partitions (Europe, Asia Pacific)

**Comparison:**
| Pattern | What You See | How It Works | Performance |
|---------|--------------|--------------|-------------|
| Dynamic Views | Only US rows | View filters at query time using `current_recipient()` | Good for moderate data |
| Partition Filtering | Only US partition | Share contains only US partition | Excellent (storage-level) |

**Key Takeaways:**
- Both patterns successfully restrict you to United States data only
- Dynamic views provide flexible logic-based filtering
- Partition filtering provides storage-level isolation with better performance

&copy; 2025 Databricks, Inc. All rights reserved.